In [ ]:
!git clone https://github.com/deepanrajm/deep_learning.git

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from pathlib import Path

In [ ]:
# Load the training data
source_data = Path("deep_learning/Generate_Text/sherlock_holmes.txt").read_text()
text = source_data.lower()

In [ ]:
# Create a list of unique characters in the data
chars = sorted(list(set(text)))
chars_to_numbers = dict((c, i) for i, c in enumerate(chars))
numbers_to_chars = dict((i, c) for i, c in enumerate(chars))

# Split the text into 40-character sequences
sequence_length = 40

# Capture both each 40-character sequence and the 41-st character that we want to predict
training_sequences = []
training_sequences_next_character = []

In [ ]:
# Loop over training text, skipping 40 characters forward on each loop
for i in range(0, len(text) - sequence_length, 40):
    # Grab the 40-character sequence as the X value
    training_sequences.append(text[i: i + sequence_length])
    # Grab the 41st character as the Y value to predict
    training_sequences_next_character.append(text[i + sequence_length])

In [ ]:
# Convert letters to numbers to make training more efficient
X = np.zeros((len(training_sequences), sequence_length, len(chars)), dtype=np.bool)
y = np.zeros((len(training_sequences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(training_sequences):
    for t, char in enumerate(sentence):
        X[i, t, chars_to_numbers[char]] = 1
    y[i, chars_to_numbers[training_sequences_next_character[i]]] = 1

In [ ]:
def generate_new_text(epoch, _):
    seed_text = "when you have eliminated the impossible,"
    new_text = ""

    # Generate 1000 characters of new text
    for i in range(1000):

        # Encode the seed text as an array of numbers the same way our training data is encoded
        x_pred = np.zeros((1, sequence_length, len(chars)))
        for t, char in enumerate(seed_text):
            x_pred[0, t, chars_to_numbers[char]] = 1.

        # Predict which letter is most likely to come next
        predicted_letter_prob = model.predict(x_pred, verbose=0)[0]

        # Uncomment these lines to control the amount of randomness.
        # # Lower values make the model less random
        # randomness = 0.6

        # Hack to prevent sum of predictions from adding up to over 1.0 due to floating point precision issues
        predicted_letter_prob *= 0.99

        # Using the letter probabilities as weights, choose the next letter randomly
        next_index = np.argmax(np.random.multinomial(1, predicted_letter_prob, 1))

        # Look up the letter itself from it's index number
        next_char = numbers_to_chars[next_index]

        # Add the new letter to our new text.
        new_text += next_char

        # Update the seed text by dropping the first letter and adding the new letter.
        # This is so we can predict the next letter in the sequence.
        seed_text = seed_text[1:] + next_char

    # Print the new text we generated
    print(new_text)

In [ ]:
inputs = keras.Input(shape=(sequence_length, len(chars)))
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(inputs)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(len(chars), activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
model.fit(X, y, batch_size=128, epochs=2,callbacks=[tf.keras.callbacks.LambdaCallback(on_epoch_end=generate_new_text)])